In [43]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [44]:
#!pip install scikit-learn

In [45]:
with open('intents.json') as file:
    data = json.load(file)

In [47]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [48]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [49]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [50]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [51]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 9)                 153       
                                                                 
Total params: 16,697
Trainable params: 16,697
Non-trainable params: 0
__________________________________________________

In [52]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
2/2 [==============================] - 0s 2ms/step - loss: 2.1962 - accuracy: 0.1389
Epoch 2/550
2/2 [==============================] - 0s 1ms/step - loss: 2.1946 - accuracy: 0.1389
Epoch 3/550
2/2 [==============================] - 0s 1ms/step - loss: 2.1936 - accuracy: 0.1389
Epoch 4/550
2/2 [==============================] - 0s 1ms/step - loss: 2.1926 - accuracy: 0.1389
Epoch 5/550
2/2 [==============================] - 0s 2ms/step - loss: 2.1915 - accuracy: 0.1389
Epoch 6/550
2/2 [==============================] - 0s 1ms/step - loss: 2.1908 - accuracy: 0.1389
Epoch 7/550
2/2 [==============================] - 0s 2ms/step - loss: 2.1901 - accuracy: 0.1389
Epoch 8/550
2/2 [==============================] - 0s 1ms/step - loss: 2.1897 - accuracy: 0.1389
Epoch 9/550
2/2 [==============================] - 0s 1ms/step - loss: 2.1891 - accuracy: 0.1944
Epoch 10/550
2/2 [==============================] - 0s 1ms/step - loss: 2.1886 - accuracy: 0.3333
Epoch 11/550
2/2 [===========

In [53]:
# saving model
model.save("chat_model.h5")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)